This notebook provides some rough examples of what can be looked at for the structured data we have. 

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import math


In [2]:
all_breaches_df = pd.read_csv("../fake_data/all_breaches_df.csv")
all_breaches_df

,offender_id,offender_name,breach_date
0,KQT6976RLP,Sian Connor,2023-04-15
1,SYZ9698SND,Mr Thomas Taylor,2023-03-10
2,COG0444UMC,Dr Debra Jackson,2023-01-28
3,IOU8015ZFN,Kenneth Baker-Jones,2023-01-10
4,BVU5951UNP,Mrs Grace Williams,2023-04-08
...,...,...,...
15422,PAJ8691NEI,Iain Brown,2023-04-23
15423,SOT1857NKE,Leslie Hughes,2023-06-01
15424,PPO2690TXG,Mrs Debra Roberts,2023-02-05
15425,REA8954BFB,Ms Michelle Williams,2023-02-03


In [3]:
pilot_tag_df = pd.read_csv("../fake_data/pilot_tag_df.csv")
pilot_tag_df

,offender_id,tag_install_date,tag_type,dob,sex,ethnic_group,original_offence
0,YEG6518QIV,2022-11-19,alcohol,1994-06-19,male,asian,Summary non-motoring
1,JME7219QND,2022-09-02,location,1986-09-19,female,white,Violence against the person
2,WSO1347WIQ,2022-10-10,curfew,1988-10-05,male,other,Drug offence
3,TUP1206QEM,2022-12-14,location,1953-12-28,male,other,Theft offence
4,AVL7422VBW,2022-08-11,alcohol,1982-04-20,female,mixed,Theft offence
...,...,...,...,...,...,...,...
1495,IUO5853OWY,2022-09-26,location,2019-04-08,female,black,Summary motoring
1496,IFV7146YSR,2022-10-16,alcohol,1901-02-06,male,unknown,Criminal Damage
1497,RON5494GDR,2022-09-08,curfew,1995-10-06,male,unknown,Summary motoring
1498,CYZ8570YTW,2022-09-19,location,1916-05-15,male,other,Summary non-motoring


## Data Quality and Initial probing

In [4]:
#what are the types of these columns?

print(all_breaches_df.dtypes)
print(pilot_tag_df.dtypes)

#"object" columns are usually strings

offender_id      object
offender_name    object
breach_date      object
dtype: object
offender_id         object
tag_install_date    object
tag_type            object
dob                 object
sex                 object
ethnic_group        object
original_offence    object
dtype: object


In [5]:
#define a function to get counts of category columns
def get_category_counts(df, column_name):
    category_counts = df[column_name].value_counts()
    return category_counts


In [6]:
get_category_counts(pilot_tag_df,"sex")

female     768
male       731
unknown      1
Name: sex, dtype: int64

In [7]:
get_category_counts(pilot_tag_df,"ethnic_group")

unknown    261
black      256
other      252
white      249
asian      248
mixed      234
Name: ethnic_group, dtype: int64

In [8]:
get_category_counts(pilot_tag_df,"original_offence")

Summary motoring               232
Drug offence                   193
Summary non-motoring           188
Other Indictable               183
Criminal Damage                179
Violence against the person    179
Theft offence                  175
Fraud                          171
Name: original_offence, dtype: int64

In [9]:
#are there any duplicate offender_ids?

def check_duplicates(df, column_name):
    duplicates = df.duplicated(subset=[column_name])
    return duplicates.any()

print(check_duplicates(all_breaches_df, 'offender_id'))
print(check_duplicates(pilot_tag_df, 'offender_id'))

#if there were duplicates, need to decide what to do with them
#can you identify which are the same person without doubt?
#if not, how much of the population does it impact? would excluding them jeopardise the analysis?

False
False


In [10]:
pilot_tag_df["dob"][0]

'1994-06-19'

In [11]:
#do a sense check on some columns

#e.g. is it ever the case that dob is smaller than tag_install_date

def remove_invalid_dates(df, date_of_birth_col_name, date_col_name):
    date_of_birth = pd.to_datetime(df[date_of_birth_col_name])
    date_col = pd.to_datetime(df[date_col_name])
    
    invalid_dates = df[date_col < date_of_birth]
    df.drop(invalid_dates.index, inplace=True)
    
    if len(invalid_dates) > 0:
        print(f"Invalid values found in column '{date_col_name}'. {len(invalid_dates)} rows removed.")
    
    return df

pilot_tag_df = remove_invalid_dates(pilot_tag_df, 'dob', 'tag_install_date')

Invalid values found in column 'tag_install_date'. 1 rows removed.


In [12]:
#are there any missing values in any of the columns?

def check_missing_values(df):
    missing_values = df.isnull().any()
    return missing_values


In [13]:

has_missing_values = check_missing_values(pilot_tag_df)
missing_columns = has_missing_values[has_missing_values].index.tolist()

if has_missing_values.any():
    print("Missing values found in the following columns:")
    print(missing_columns)
else:
    print("No missing values found in any columns.")

No missing values found in any columns.


In [14]:

has_missing_values = check_missing_values(all_breaches_df)
missing_columns = has_missing_values[has_missing_values].index.tolist()

if has_missing_values.any():
    print("Missing values found in the following columns:")
    print(missing_columns)
else:
    print("No missing values found in any columns.")

No missing values found in any columns.


## Some exploratory analysis

In [15]:
pilot_tag_df.dtypes

offender_id         object
tag_install_date    object
tag_type            object
dob                 object
sex                 object
ethnic_group        object
original_offence    object
dtype: object

In [16]:
#make an age_at_tag column (what age the offender was when they got the tag installed)

# Convert string dates to datetime objects
pilot_tag_df['dob'] = pd.to_datetime(pilot_tag_df['dob'])
pilot_tag_df['tag_install_date'] = pd.to_datetime(pilot_tag_df['tag_install_date'])

# Calculate the difference in years
pilot_tag_df['age_at_tag_install'] = (pilot_tag_df['tag_install_date'] - pilot_tag_df['dob']).dt.days / 365
pilot_tag_df['age_at_tag_install'] = pilot_tag_df['age_at_tag_install'].astype(int)
#pilot_tag_df
pilot_tag_df['age_at_tag_install'].describe()

count    1499.000000
mean       59.160107
std        35.288243
min         0.000000
25%        28.000000
50%        57.000000
75%        90.000000
max       122.000000
Name: age_at_tag_install, dtype: float64

In [17]:
#filter out any rows with age_at_tag_install less than 18 years old
#possibly erroneous
pilot_tag_df = pilot_tag_df[pilot_tag_df["age_at_tag_install"]>17].reset_index(drop=True)
pilot_tag_df

,offender_id,tag_install_date,tag_type,dob,sex,ethnic_group,original_offence,age_at_tag_install
0,YEG6518QIV,2022-11-19,alcohol,1994-06-19,male,asian,Summary non-motoring,28
1,JME7219QND,2022-09-02,location,1986-09-19,female,white,Violence against the person,35
2,WSO1347WIQ,2022-10-10,curfew,1988-10-05,male,other,Drug offence,34
3,TUP1206QEM,2022-12-14,location,1953-12-28,male,other,Theft offence,69
4,AVL7422VBW,2022-08-11,alcohol,1982-04-20,female,mixed,Theft offence,40
...,...,...,...,...,...,...,...,...
1274,TLE7766TYO,2022-08-10,curfew,1961-09-26,male,asian,Fraud,60
1275,IFV7146YSR,2022-10-16,alcohol,1901-02-06,male,unknown,Criminal Damage,121
1276,RON5494GDR,2022-09-08,curfew,1995-10-06,male,unknown,Summary motoring,26
1277,CYZ8570YTW,2022-09-19,location,1916-05-15,male,other,Summary non-motoring,106


In [18]:
#it is optional here if you want to remove outliers
#although in real analyses you'd probably want to consider what an outlier means in your data
#is it actually an outlier, what would the impact be of removing it

In [19]:
#assign age groups

# Calculate the maximum value in the 'YearDiff' column
max_value = math.ceil(pilot_tag_df['age_at_tag_install'].max() / 10) * 10

# Define the category ranges
category_ranges = range(10, max_value + 10, 10)  # Creates ranges of 10-20, 20-30, etc.

# Create the categories using pd.cut()
pilot_tag_df['age_group'] = pd.cut(pilot_tag_df['age_at_tag_install'], bins=category_ranges, right=False)

# Adjust the category labels to represent the corresponding range as strings
pilot_tag_df['age_group'] = pilot_tag_df['age_group'].apply(lambda x: f'{x.left + 1}-{x.right}')


pilot_tag_df["age_group"].unique()

[21-30, 31-40, 61-70, 41-50, 51-60, ..., 81-90, 91-100, 101-110, 121-130, 11-20]
Length: 12
Categories (12, object): [11-20 < 21-30 < 31-40 < 41-50 ... 91-100 < 101-110 < 111-120 < 121-130]

In [20]:
pilot_tag_df

,offender_id,tag_install_date,tag_type,dob,sex,ethnic_group,original_offence,age_at_tag_install,age_group
0,YEG6518QIV,2022-11-19,alcohol,1994-06-19,male,asian,Summary non-motoring,28,21-30
1,JME7219QND,2022-09-02,location,1986-09-19,female,white,Violence against the person,35,31-40
2,WSO1347WIQ,2022-10-10,curfew,1988-10-05,male,other,Drug offence,34,31-40
3,TUP1206QEM,2022-12-14,location,1953-12-28,male,other,Theft offence,69,61-70
4,AVL7422VBW,2022-08-11,alcohol,1982-04-20,female,mixed,Theft offence,40,41-50
...,...,...,...,...,...,...,...,...,...
1274,TLE7766TYO,2022-08-10,curfew,1961-09-26,male,asian,Fraud,60,61-70
1275,IFV7146YSR,2022-10-16,alcohol,1901-02-06,male,unknown,Criminal Damage,121,121-130
1276,RON5494GDR,2022-09-08,curfew,1995-10-06,male,unknown,Summary motoring,26,21-30
1277,CYZ8570YTW,2022-09-19,location,1916-05-15,male,other,Summary non-motoring,106,101-110


In [21]:
#create a column in pilot_tag_df based on if an offender_id has breached
#do this via a left join 

pilot_tag_df = pilot_tag_df.merge(all_breaches_df,how="left",on="offender_id")
pilot_tag_df
#in real life, getting information from joining tables is very common
#you're lucky if you have a data modeller who has mapped out all the connections
#between columns in tables, otherwise you have to do that painful task yourself!
#usually table joins aren't as pretty as this one in our example!


#after this join you could do some double checking to see if the offender_name seems to match the sex, for example

,offender_id,tag_install_date,tag_type,dob,sex,ethnic_group,original_offence,age_at_tag_install,age_group,offender_name,breach_date
0,YEG6518QIV,2022-11-19,alcohol,1994-06-19,male,asian,Summary non-motoring,28,21-30,Ricky Booth,2023-03-11
1,JME7219QND,2022-09-02,location,1986-09-19,female,white,Violence against the person,35,31-40,Dr Charlene Jordan,2023-03-05
2,WSO1347WIQ,2022-10-10,curfew,1988-10-05,male,other,Drug offence,34,31-40,Joseph Green,2023-03-23
3,TUP1206QEM,2022-12-14,location,1953-12-28,male,other,Theft offence,69,61-70,Mr Danny Robinson,2023-02-13
4,AVL7422VBW,2022-08-11,alcohol,1982-04-20,female,mixed,Theft offence,40,41-50,Miss Mary Taylor,2023-02-01
...,...,...,...,...,...,...,...,...,...,...,...
1274,TLE7766TYO,2022-08-10,curfew,1961-09-26,male,asian,Fraud,60,61-70,NaN,NaN
1275,IFV7146YSR,2022-10-16,alcohol,1901-02-06,male,unknown,Criminal Damage,121,121-130,NaN,NaN
1276,RON5494GDR,2022-09-08,curfew,1995-10-06,male,unknown,Summary motoring,26,21-30,NaN,NaN
1277,CYZ8570YTW,2022-09-19,location,1916-05-15,male,other,Summary non-motoring,106,101-110,NaN,NaN


In [22]:
pilot_tag_df["breach"] =  np.where(pilot_tag_df['breach_date'].notna(), 'Yes', "No")
#check counts
get_category_counts(pilot_tag_df,"breach")

No     1130
Yes     149
Name: breach, dtype: int64

In [23]:
#% of tag wearers who breached within the time
(149/(1130+149))*100

11.64972634870993

In [24]:
#column difference in date got tag and date that breached
pilot_tag_df['breach_date'] = pd.to_datetime(pilot_tag_df['breach_date'])

# Calculate the difference in days
pilot_tag_df['breach_time'] = (pilot_tag_df['breach_date'] - pilot_tag_df['tag_install_date']).dt.days

#stats for the column
pilot_tag_df['breach_time'].describe()

count    149.000000
mean     156.476510
std       61.968015
min       15.000000
25%      114.000000
50%      158.000000
75%      197.000000
max      303.000000
Name: breach_time, dtype: float64

In [25]:
#% breached by category
def calculate_percentage(df, group_col):
    # Group the dataframe by the specified column
    grouped = df.groupby(group_col)

    # Count the occurrences of 'Yes' in 'breach_time' for each group
    group_counts = grouped['breach'].apply(lambda x: (x == 'Yes').sum())
    #print(group_counts)

    # Calculate the total count of 'offender_id' for each group
    total_counts = grouped['offender_id'].count()
    #print(total_counts)
    # Calculate the percentage for each group
    percentages = (group_counts / total_counts) * 100

    return percentages


In [26]:
pilot_tag_df.dtypes

offender_id                   object
tag_install_date      datetime64[ns]
tag_type                      object
dob                   datetime64[ns]
sex                           object
ethnic_group                  object
original_offence              object
age_at_tag_install             int32
age_group                   category
offender_name                 object
breach_date           datetime64[ns]
breach                        object
breach_time                  float64
dtype: object

In [27]:
calculate_percentage(pilot_tag_df, 'tag_type')

tag_type
alcohol     10.117647
curfew      13.270142
location    11.574074
dtype: float64

In [28]:
calculate_percentage(pilot_tag_df, 'sex')
#unknown likely anomalous as only 1 unknown record so appears like 100%

sex
female      10.093168
male        13.091483
unknown    100.000000
dtype: float64

In [29]:
calculate_percentage(pilot_tag_df, 'age_group')

age_group
11-20       6.451613
21-30      11.764706
31-40      15.909091
41-50      11.200000
51-60      10.526316
61-70      16.666667
71-80       9.917355
81-90       9.803922
91-100     11.403509
101-110    11.607143
111-120     9.243697
121-130     8.823529
dtype: float64

In [30]:
calculate_percentage(pilot_tag_df, 'ethnic_group')

ethnic_group
asian      11.483254
black      13.063063
mixed      11.675127
other      13.145540
unknown    11.013216
white       9.478673
dtype: float64

In [31]:
calculate_percentage(pilot_tag_df, 'original_offence')

original_offence
Criminal Damage                 8.163265
Drug offence                   10.625000
Fraud                          12.500000
Other Indictable               14.012739
Summary motoring               13.170732
Summary non-motoring            6.790123
Theft offence                  14.666667
Violence against the person    12.987013
dtype: float64

In [32]:
#we can use chi sq tests of independance to 
#see if there is an association between the categorical variables
#of "breach" and the rest

#in real life you might 
#1) check if you meet the assumptions of the test e.g. if requires normality, might need non-parametric version of test
#2) check if there are other tests avaiable
#3 Consider if the frequentist approach to testing is not vulnerable in this context compared to Bayesian version of tests. 
#  Frequentist tests usually become more likely to be significant are the dataset size increases. 

In [33]:
from scipy.stats import chi2_contingency

def calculate_chi_squared(df, column_one,column_two, sig_lev):
    # Create a contingency table between the two columns
    contingency_table = pd.crosstab(df[column_one], df[column_two])

    print(contingency_table)
    #you could additionally check to see which exact cells in contingency_table are causing the
    #significance at your desired level e.g. if value is 2sd above mean
    
    # Perform chi-squared test
    chi2, p_value, _, _ = chi2_contingency(contingency_table)
    
    print("Chi-squared:", chi2)
    print("P-value:", p_value)
    
    test_result = ""
    
    if p_value < sig_lev:test_result = "significant"
    else: test_result = "not significant"

    return test_result

In [34]:
# 0.05 p-value is "conventional" in academia, but there have been other papers which
# have argued to reducing it a lot e.g. to 0.001 
calculate_chi_squared(pilot_tag_df, 'breach', 'tag_type', 0.05)

tag_type  alcohol  curfew  location
breach                             
No            382     366       382
Yes            43      56        50
Chi-squared: 2.0482030609454913
P-value: 0.35911897795876024


'not significant'

In [35]:
calculate_chi_squared(pilot_tag_df, 'breach', 'sex', 0.05)
#this test shouldn't actually be done because one of the assumptions of chi sq test is that no cells have
#vales below a count of 5 (and the unknown sex values do)
#this is why it's really important to check the analytical technique you're choosing to use is appropriate

sex     female  male  unknown
breach                       
No         579   551        0
Yes         65    83        1
Chi-squared: 10.380282609027894
P-value: 0.005571219520830717


'significant'

In [36]:
calculate_chi_squared(pilot_tag_df, 'breach', 'age_group', 0.05)

age_group  11-20  21-30  31-40  41-50  51-60  61-70  71-80  81-90  91-100  \
breach                                                                      
No            29    120    111    111    119    100    109     92     101   
Yes            2     16     21     14     14     20     12     10      13   

age_group  101-110  111-120  121-130  
breach                                
No              99      108       31  
Yes             13       11        3  
Chi-squared: 7.894938571303869
P-value: 0.7226830564615045


'not significant'

In [37]:
calculate_chi_squared(pilot_tag_df, 'breach', 'ethnic_group', 0.05)

ethnic_group  asian  black  mixed  other  unknown  white
breach                                                  
No              185    193    174    185      202    191
Yes              24     29     23     28       25     20
Chi-squared: 1.9552542176727514
P-value: 0.8553011944212996


'not significant'

In [38]:
calculate_chi_squared(pilot_tag_df, 'breach', 'original_offence', 0.05)


original_offence  Criminal Damage  Drug offence  Fraud  Other Indictable  \
breach                                                                     
No                            135           143    126               135   
Yes                            12            17     18                22   

original_offence  Summary motoring  Summary non-motoring  Theft offence  \
breach                                                                    
No                             178                   151            128   
Yes                             27                    11             22   

original_offence  Violence against the person  
breach                                         
No                                        134  
Yes                                        20  
Chi-squared: 8.624016664884548
P-value: 0.2807885651098836


'not significant'

In [39]:
#in your own time, you can can do other tests like checking if time to breach significantly differs by categories of offender

## Naive linear forecasts

In [40]:
#forecasting future breached offenders from pilot
#this is a very naive forecast with huge uncertainty
#because it only takes into consider the count of those who breached

#need to use all_breaches_df
#add col to see if that offender was in the pilot 